In [1]:
import glob
import pathlib
import pickle
import numpy as np
import sys
import os
from music21 import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

"""
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras import applications
from keras import datasets
from keras import preprocessing
from keras import wrappers
from keras_self_attention import SeqSelfAttention # requires to use keras directly without tf """


sys.path.append("..")
from musiclearn import config

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Data Processing and Exploration

In [2]:
# path to Schubert collection
schubert_dir = pathlib.Path(config.MUSICNET_MIDI_DIR) / "Schubert"

# path to final fantasy collection
ff_dir = pathlib.Path(config.FF_MIDI_DIR)

# path to location of tensorboard logs
logs_dir = pathlib.Path(config.LOGS_DIR)

In [3]:
# length of an input sequence
SEQUENCE_LENGTH = 100

In [4]:
def get_notes(midi_dir):
    """ Get all the notes and chords from the midi files in a collection """
    notes = []
    
    noteFile_dir = str(midi_dir) + "/notes"
    
    if not os.path.exists(noteFile_dir):
        files_dir = str(midi_dir) + "/*.mid"
        for file in glob.glob(files_dir):
            midi = converter.parse(file)

            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse()
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                     notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        # save the features
        with open(noteFile_dir, 'wb') as fp:
            pickle.dump(notes, fp)
    else:
        with open(noteFile_dir, 'rb') as fp:
            notes = pickle.load(fp)
    
    return notes

In [5]:
def read_midi(midi_dir):
    """ Read all midi file in the collection and return the notes """
    notes = []
    
    noteFile_dir = str(midi_dir) + "/notes"
    
    if not os.path.exists(noteFile_dir):
        files_dir = str(midi_dir) + "/*.mid"
        for file in glob.glob(files_dir):
            midi = converter.parse(file)

            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                
                # looping over all the instrument parts
                for part in s2.parts:
                    #select elements of piano
                    if "Piano" in str(part):
                        notes_to_parse = part.recurse()
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            # if there are notes to parse
            if notes_to_parse is not None:
                for element in notes_to_parse:
                    if isinstance(element, note.Note):
                         notes.append(str(element.pitch))
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))

        # save the features
        with open(noteFile_dir, 'wb') as fp:
            pickle.dump(notes, fp)
    else:
        with open(noteFile_dir, 'rb') as fp:
            notes = pickle.load(fp)
    
    return notes

In [6]:
# extract the notes
#notes = get_notes(schubert_dir)
notes = read_midi(schubert_dir)

In [7]:
# get the number of unique notes
pitchNames = sorted(set(item for item in notes))
n_vocab = len(pitchNames)
print("No. unique notes =", n_vocab)

No. unique notes = 312


In [8]:
pitchNames

['0',
 '0.1',
 '0.1.6',
 '0.2',
 '0.2.5',
 '0.2.5.8',
 '0.2.6',
 '0.2.6.8',
 '0.2.7',
 '0.3',
 '0.3.4',
 '0.3.5',
 '0.3.6',
 '0.3.6.8',
 '0.3.6.9',
 '0.3.7',
 '0.4',
 '0.4.5',
 '0.4.6',
 '0.4.7',
 '0.4.8',
 '0.5',
 '0.6',
 '1',
 '1.2',
 '1.2.6',
 '1.3',
 '1.3.6',
 '1.3.6.9',
 '1.3.7',
 '1.3.8',
 '1.4',
 '1.4.6',
 '1.4.7',
 '1.4.7.10',
 '1.4.7.9',
 '1.4.7.9.10',
 '1.4.8',
 '1.5',
 '1.5.7',
 '1.5.8',
 '1.5.9',
 '1.6',
 '1.7',
 '10',
 '10.0',
 '10.0.2',
 '10.0.3',
 '10.0.4',
 '10.0.5',
 '10.1',
 '10.1.3',
 '10.1.4',
 '10.1.4.5',
 '10.1.4.6',
 '10.1.5',
 '10.2',
 '10.2.3',
 '10.2.4',
 '10.2.5',
 '10.3',
 '11',
 '11.0',
 '11.0.4',
 '11.1',
 '11.1.4',
 '11.1.4.6',
 '11.1.4.7',
 '11.1.5',
 '11.1.6',
 '11.2',
 '11.2.4',
 '11.2.4.7',
 '11.2.5',
 '11.2.5.7',
 '11.2.6',
 '11.3',
 '11.3.5',
 '11.3.6',
 '11.4',
 '2',
 '2.3',
 '2.3.7',
 '2.4',
 '2.4.6',
 '2.4.7',
 '2.4.8',
 '2.4.8.9',
 '2.4.9',
 '2.5',
 '2.5.7',
 '2.5.8',
 '2.5.8.10',
 '2.5.8.11',
 '2.5.8.9',
 '2.5.9',
 '2.6',
 '2.6.10',
 '2.6.8',
 

In [9]:
# create a dictionary to map notes to unique integers
note_to_int = dict((note, number) for number, note in enumerate(pitchNames))

# create a dictionary to map unique integers to notes
int_to_note = dict((number, note) for number, note in enumerate(pitchNames))

In [10]:
def prepare_sequences(sequence_length, notes, n_vocab, note_to_int):
    """ Prepare the sequences used by the neural network """   
    network_input = []
    network_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_samples = len(network_input)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_samples, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = np.array(network_output)
    
    return (network_input, network_output)

In [11]:
# test prepare_sequences()
network_input, network_output = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
print(network_input.shape)
print(network_output.shape)

(78937, 100, 1)
(78937,)


In [12]:
def create_midi(notes_list, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in notes_list:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=filename)

In [13]:
def train_lstm_model(model, sequence_length, notes, n_vocab, note_to_int, model_name, epochs=20, batch_size=128):
    """ Train a neural network to generate music """
    # get amount of pitch names
    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(sequence_length, notes, n_vocab, note_to_int)

    #model = lstm_model(network_input.shape, n_vocab)
    
    # train the network
    filepath = model_name + "_saved_weight.hdf5"
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        node='min'
    )
    
    log_dir = str(logs_dir / "logs" / model_name)
    tensorboard_cb = TensorBoard(log_dir=log_dir,
                                     write_graph=True,
                                     write_images=True,
                                     update_freq='epoch'
                                )
    
    callback_list = [checkpoint_cb, tensorboard_cb]
    
    model.fit(network_input, 
              network_output, 
              epochs=epochs, 
              batch_size=batch_size, 
              callbacks=callback_list,
              validation_split=0.2
             )

In [14]:
def generate_notes(model, network_input, int_to_note, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        #pattern.append(index)
        pattern = np.append(pattern, [index])
        pattern = pattern[1:]

    return prediction_output

## LSTM Model

In [ ]:
def lstm_model(input_shape, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(input_shape[1], input_shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add((LSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [ ]:
K.clear_session()

In [ ]:
# test code to output model architecture
network_input, network_output = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
lstm_model = lstm_model(network_input.shape, n_vocab)
lstm_model.summary()

In [ ]:
# train the model
train_lstm_model(lstm_model, SEQUENCE_LENGTH, notes, n_vocab, note_to_int, "lstm", 100, 128)

In [ ]:
def load_lstm_model():
    """ Load the saved LSTM model """
    
    model = tf.keras.models.load_model('lstm_saved_weight.hdf5')
    
    return model

In [ ]:
# generate a new midi file from the train model
noteFile_dir = str(schubert_dir / "notes")
    
# load the notes used to train the model
with open(noteFile_dir, 'rb') as fp:
    notes = pickle.load(fp)

network_input, _ = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
print(network_input[0].shape)
    
# load saved weights
model = load_lstm_model()

# generate a new sequence
prediction_output = generate_notes(model, network_input, int_to_note, n_vocab)

 # make midi file
create_midi(prediction_output, "lstm_output_sample.mid")

## Bidirectional LSTM

In [ ]:
def bidirectional_lstm_model(input_shape, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(input_shape[1], input_shape[2])))
    
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(512)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [ ]:
K.clear_session()

In [ ]:
# test code to output model architecture
network_input, network_output = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
bidirectional_lstm_model = bidirectional_lstm_model(network_input.shape, n_vocab)
bidirectional_lstm_model.summary()

In [ ]:
# train the model
train_lstm_model(bidirectional_lstm_model, SEQUENCE_LENGTH, notes, n_vocab, note_to_int, "bidirect_lstm", 100, 128)

In [ ]:
def load_bidirectional_lstm_model():
    """ Load the saved LSTM model """
    
    model = tf.keras.models.load_model('bidirect_lstm_saved_weight.hdf5')
    
    return model

In [ ]:
# generate a new midi file from the train model
noteFile_dir = str(schubert_dir / "notes")
    
# load the notes used to train the model
with open(noteFile_dir, 'rb') as fp:
    notes = pickle.load(fp)

network_input, _ = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
print(network_input[0].shape)
    
# load saved weights
model = load_bidirectional_lstm_model()

# generate a new sequence
prediction_output = generate_notes(model, network_input, int_to_note, n_vocab)

 # make midi file
create_midi(prediction_output, "bidirect_lstm_output_sample.mid")

## LSTM with Attention

In [15]:
class Customized_Attention(Layer):
    """ Attention block """
    def __init__(self, return_sequences=True):
        self.return_sequences=return_sequences
        super().__init__()
        
        
    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        super().build(input_shape)
        
        
    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        
        if self.return_sequences:
            return output
        return K.sum(output, axis=1)
    
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({'return_sequences': self.return_sequences})
        return config
    

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [16]:
def attention_lstm_model(input_shape, n_vocab):
    """ Construct a model based on LSTM and attention """
    model = Sequential()
    
    # Bidirectional LSTM layer with attention
    model.add(Bidirectional(LSTM(512, return_sequences=True), 
                            input_shape=(input_shape[1], input_shape[2])
                           )
             )
    model.add(Customized_Attention(return_sequences=True))
    model.add(Dropout(0.3))
    
    # Second bidirectional LSTM layer with attention
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Customized_Attention(return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    # compile
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [17]:
K.clear_session()

In [18]:
# test the architecture
network_input, network_output = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
attention_lstm_model = attention_lstm_model(network_input.shape, n_vocab)
attention_lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 1024)         2105344   
_________________________________________________________________
customized__attention (Custo (None, 100, 1024)         1124      
_________________________________________________________________
dropout (Dropout)            (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 1024)         6295552   
_________________________________________________________________
customized__attention_1 (Cus (None, 100, 1024)         1124      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6

In [ ]:
plot_model(attention_lstm_model)

In [ ]:
# train the model
train_lstm_model(attention_lstm_model, SEQUENCE_LENGTH, notes, n_vocab, note_to_int, "attention_lstm", 100, 128)

In [19]:
def load_attention_lstm_model(input_shape, n_vocab):
    """ Load the saved weights """
    #model = tf.keras.models.load_model("attention_lstm_saved_weight.hdf5")
    
    model = Sequential()
    
    # Bidirectional LSTM layer with attention
    model.add(Bidirectional(LSTM(512, return_sequences=True), 
                            input_shape=(input_shape[1], input_shape[2])
                           )
             )
    model.add(Customized_Attention(return_sequences=True))
    model.add(Dropout(0.3))
    
    # Second bidirectional LSTM layer with attention
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Customized_Attention(return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    # compile
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
    
    model.load_weights("attention_lstm_saved_weight.hdf5")
    
    return model

In [20]:
# generate a new midi file from the train model
noteFile_dir = str(schubert_dir / "notes")
    
# load the notes used to train the model
with open(noteFile_dir, 'rb') as fp:
    notes = pickle.load(fp)

network_input, _ = prepare_sequences(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
print(network_input[0].shape)
    
# load saved weights
model = load_attention_lstm_model(network_input.shape, n_vocab)

# generate a new sequence
prediction_output = generate_notes(model, network_input, int_to_note, n_vocab)

 # make midi file
create_midi(prediction_output, "attention_lstm_output_sample.mid")

(100, 1)


## WaveNet model

In [ ]:
K.clear_session()

In [ ]:
def simplified_wavenet(n_vocab, sequence_length):
    """ A simplified version of WaveNet without residual and skip connection """
    model = Sequential()
    
    # embedding layer
    model.add(Embedding(n_vocab, 100, input_length=sequence_length))
    
    # causal convolution
    model.add(Conv1D(64, 3, padding='causal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    
    # dialated causal convolution
    model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))

    model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    
    #model.add(Conv1D(256,5,activation='relu'))    
    model.add(GlobalMaxPool1D())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_vocab, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    
    return model

In [ ]:
def WaveNetResidualConv1D(num_filters, kernel_size, dilation_rate):
    """ Function that creates a residual block for the WaveNet with gated
        activation units, skip connections and residual output, as described
        in Sections 2.3 and 2.4 of the paper [1].
        Args:
            num_filters (int): Number of filters used for convolution.
            kernel_size (int): The size of the convolution.
            dilation_rate (int): The dilation rate for the dilated convolution.
        Returns:
            A layer wrapper compatible to the Keras functional API.
        See:
            [1] Oord, Aaron van den, et al. "Wavenet: A generative model for
                raw audio." arXiv preprint arXiv:1609.03499 (2016).
    """
    def build_residual_block(l_input):
        # Gated activation.
        l_sigmoid_conv1d = Conv1D(num_filters, kernel_size, dilation_rate=dilation_rate, 
                                  padding="same", activation="sigmoid")(l_input)
        l_tanh_conv1d = Conv1D(num_filters, kernel_size, dilation_rate=dilation_rate,
                               padding="same", activation="tanh")(l_input)
        l_mul = Multiply()([l_sigmoid_conv1d, l_tanh_conv1d])
        # Branches out to skip unit and residual output.
        l_skip_connection = Conv1D(1, 1)(l_mul)
        l_residual = Add()([l_input, l_skip_connection])
        return l_residual, l_skip_connection
    
    return build_residual_block


def build_wavenet_model(input_size, num_filters, kernel_size,
                        num_residual_blocks):
    """ Returns an implementation of WaveNet, as described in Section 2
        of the paper [1].
        Args:
            input_size (int): The size of the waveform the network will
                consider as input.
            num_filters (int): Number of filters used for convolution.
            kernel_size (int): The size of the convolution.
            num_residual_blocks (int): How many residual blocks to generate
                between input and output. Residual block i will have a dilation
                rate of 2^(i+1), i starting from zero.
        Returns:
            A Keras model representing the WaveNet.
        See:
            [1] Oord, Aaron van den, et al. "Wavenet: A generative model for
                raw audio." arXiv preprint arXiv:1609.03499 (2016).
    """
    l_input = Input(batch_shape=(None, input_size, 1))
    l_stack_conv1d = Conv1D(num_filters, kernel_size, padding="same")(l_input)
    l_skip_connections = []
    for i in range(num_residual_blocks):
        l_stack_conv1d, l_skip_connection = WaveNetResidualConv1D(
            num_filters, kernel_size, 2 ** (i + 1))(l_stack_conv1d)
        l_skip_connections.append(l_skip_connection)
    l_sum = Add()(l_skip_connections)
    relu = Activation("relu")(l_sum)
    l1_conv1d = Conv1D(1, 1, activation="relu")(relu)
    l2_conv1d = Conv1D(1, 1)(l1_conv1d)
    l_flatten = Flatten()(l2_conv1d)
    l_output = Dense(256, activation="softmax")(l_flatten)
    model = Model(inputs=[l_input], outputs=[l_output])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [ ]:
wavenet_model = simplified_wavenet(n_vocab, SEQUENCE_LENGTH)
wavenet_model.summary()

In [ ]:
def prepare_sequences_wavenet(sequence_length, notes, n_vocab, note_to_int):
    """ Prepare the sequences used by the neural network """   
    network_input = []
    network_output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_samples = len(network_input)
    
    # no reshape
    network_input = np.array(network_input)
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = np.array(network_output)
    
    return (network_input, network_output)

In [ ]:
# prepare the input to the model
X, Y = prepare_sequences_wavenet(SEQUENCE_LENGTH, notes, n_vocab, note_to_int)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
checkpoint_cb = ModelCheckpoint(
    "wavenet_weights.hdf5", 
    monitor='val_loss', 
    mode='min',
    save_best_only=True,
    verbose=1
    )

log_dir = str(logs_dir / "logs" / 'wavenet')
tensorboard_cb = TensorBoard(log_dir=log_dir,
                                 write_graph=True,
                                 write_images=True,
                                 update_freq='epoch'
                            )

history = wavenet_model.fit(X_train,
                    Y_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(X_val, Y_val),
                    verbose=1,
                    callbacks=[checkpoint_cb, tensorboard_cb]
                   )

In [ ]:
def load_wavenet_model():
    """ Load the saved weights of the WaveNet model """
    model = tf.keras.models.load_model('wavenet_weights.hdf5')
    
    return model

In [ ]:
# load the trained model
wavenet_model = load_wavenet_model()

In [ ]:
def generate_notes(model, network_input, int_to_note, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    pattern = list(network_input[start])
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern)))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:]

    return prediction_output

In [ ]:
# generate new sequence with wavenet
predicted_notes = generate_notes(wavenet_model, X_val, int_to_note, n_vocab)

# convert to MiDi file
create_midi(predicted_notes, "wavenet_sample_output.mid")